In [1]:
# import library
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import SimpleRNN, LSTM, GRU
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K
from keras.utils import multi_gpu_model
from keras import metrics
from keras import losses
from keras import __version__
print('Using Keras version:', __version__, 'backend:', K.backend())
from keras.backend.tensorflow_backend import set_session

from sklearn.model_selection import GridSearchCV
# from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.metrics import classification_report # https://stackoverflow.com/questions/50065484/getting-precision-recall-and-f1-score-per-class-in-keras

from IPython.display import Javascript
import numpy as np
from distutils.version import LooseVersion as LV
from datetime import datetime
import csv
import sys
from os import listdir
from os.path import isfile, join
import pprint
#import boto3
import pickle
import time
import os.path
import pickle
sys.path.append(os.getcwd())
from matplotlib import pyplot

import smtplib
from email.mime.text import MIMEText # simple MSG
from email.mime.multipart import MIMEMultipart # complex MSG
        
from link_aws_key import *
from email_info import *
pp = pprint.PrettyPrinter(indent=4)

import warnings
warnings.filterwarnings("ignore")

coins = {
    0: 'KRW',
    1: 'BTC',
    2: 'ETH',
    3: 'XRP',
    4: 'BCH',
    5: 'LTC',
    6: 'DASH',
    7: 'ETC'
}

# aws_client = boto3.client(
#     's3',
#     aws_access_key_id=LINK_AWSAccessKeyId,
#     aws_secret_access_key=LINK_AWSSecretKey
# )

bucket = "bithumb10"
cleanup_file_name = "coin_{0}_{1}_cleanup.csv"

/home/link/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Using Keras version: 2.2.2 backend: tensorflow


In [2]:
def get_all_raw_data_from_aws(coin_name_list, start_date, end_date):
    start_ms_time = datetime.strptime(start_date + " +0900", "%Y-%m-%d %H:%M:%S %z").timestamp() * 1000
    end_ms_time = datetime.strptime(end_date + " +0900", "%Y-%m-%d %H:%M:%S %z").timestamp() * 1000
    
    year_temp = start_date[:4]
    years = [year_temp]
    while year_temp < end_date[:4]:
        year_temp = str(int(start_date[:4]) + 1)
        years.append(year_temp)
    raw_data = {}  # 전체 CSV Raw 데이터
    for coin_name in coin_name_list:
        raw_data[coin_name] = []

    # KRW 제외한 나머지 CSV Raw 데이터 수집
    for coin_name in coin_name_list:
        if coin_name == 'KRW':
            continue
        lines = []
        for year in years:
            obj = aws_client.get_object(
                Bucket=bucket,
                Key='cleanup/' + year + '/' + cleanup_file_name.format(coin_name, year)
            )
            if lines != []:
                lines += obj.get('Body')._raw_stream.readlines()
            else:
                lines = obj.get('Body')._raw_stream.readlines()

        for line in lines:
            line = str(line.strip())[2:-1]
            line = line.split(',')
            if start_ms_time <= int(line[0]) and int(line[0]) <= end_ms_time:
                raw_data[coin_name].append(line)

    raw_data['KRW'] = list()
    for line in raw_data['BTC']:
        raw_data['KRW'].append([line[0], line[1], 1, 1, 1, 1, 1.0, 'normal'])

    return raw_data

def get_all_raw_data(coin_name_list, start_date, end_date, data_files_dir):
    trading_files = []
    for coin_name in coin_name_list:
        for data_file_name in [f for f in listdir(data_files_dir) if isfile(join(data_files_dir, f))]:
            if coin_name in data_file_name:
                trading_files.append(data_file_name)

    start_ms_time = datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S").timestamp() * 1000
    end_ms_time = datetime.strptime(end_date, "%Y-%m-%d %H:%M:%S").timestamp() * 1000
    
    raw_data = {} #전체 CSV Raw 데이터
    for coin_name in coin_name_list:
        raw_data[coin_name] = []
    
    #KRW 제외한 나머지 CSV Raw 데이터 수집
    for coin_name in coin_name_list:
        for data_file_name in trading_files:
            if coin_name in data_file_name:
                file = open(data_files_dir + data_file_name, 'r', encoding='utf-8')
                rdr = csv.reader(file)
                for line in rdr:
                    if start_ms_time <= int(line[0]) and int(line[0]) <= end_ms_time:
                        raw_data[coin_name].append(line)
                file.close()
    
    for line in raw_data['BTC']:
        raw_data['KRW'].append([line[0], line[1], 1, 1, 1, 1, 1.0, 'normal'])
#     print("test")
    return raw_data
    
def make_cryptocurrency_dataset_X(coin_name_list, start_date, end_date, time_unit, window_size, gap, margin_rate, data_files_dir):
    num_coins = len(coin_name_list)
    #raw_data = get_all_raw_data_from_aws(coin_name_list, start_date, end_date)
    raw_data = get_all_raw_data(coin_name_list, start_date, end_date, data_files_dir)    
    num_sticks = len(raw_data['BTC'])
    
    if time_unit % 10 != 0 or num_sticks < (time_unit / 10) * window_size + gap:
        return None, None
    
    num = int(num_sticks - ((time_unit / 10) * window_size + gap) + 1)
        
    X = []
    # (윈도우 개수, 코인 개수, 윈도우 사이즈, 3)
    for idx in range(num):
        X.append([])
        idx_coin = 0
        for coin_name in coin_name_list:
            X[idx].append([])
            
            last_idx_in_window = int(idx + time_unit / 10 * window_size - 1)
            close_price_in_last_idx_in_window = float(raw_data[coin_name][last_idx_in_window][3])
            
            for idx_in_window in range(window_size):
                X[idx][idx_coin].append([])
                idx_stick = int(idx + time_unit / 10 * (idx_in_window + 1) - 1)
                X[idx][idx_coin][idx_in_window].append(float(raw_data[coin_name][idx_stick][3]) / close_price_in_last_idx_in_window)
                X[idx][idx_coin][idx_in_window].append(float(raw_data[coin_name][idx_stick][4]) / close_price_in_last_idx_in_window)
                X[idx][idx_coin][idx_in_window].append(float(raw_data[coin_name][idx_stick][5]) / close_price_in_last_idx_in_window)
                X[idx][idx_coin][idx_in_window].append(float(raw_data[coin_name][idx_stick][6]))
                
    X = np.array(X)
    
    return X

def make_cryptocurrency_dataset_y(coin_name_list, start_date, end_date, time_unit, window_size, gap, margin_rate, data_files_dir):
    y_trv = []
    y_btv = []
    num_coins = len(coin_name_list)
    #raw_data = get_all_raw_data_from_aws(coin_name_list, start_date, end_date)
    raw_data = get_all_raw_data(coin_name_list, start_date, end_date, data_files_dir)    
    num_sticks = len(raw_data['BTC'])
    
    if time_unit % 10 != 0 or num_sticks < (time_unit / 10) * window_size + gap:
        return None, None
    
    num = int(num_sticks - ((time_unit / 10) * window_size + gap) + 1)
        
    
    y = []
    # (윈도우 개수, 코인 개수, 윈도우 사이즈, 3)
    for idx in range(num):
        
        y.append([])
        idx_coin = 0
        for coin_name in coin_name_list:            
            last_idx_in_window = int(idx + time_unit / 10 * window_size - 1)
            close_price_in_last_idx_in_window = float(raw_data[coin_name][last_idx_in_window][3])
            target_idx_for_window = int(idx + time_unit / 10 * window_size - 1 + gap)
            target_price = float(raw_data[coin_name][target_idx_for_window][3])
            target = 0
            
            if target_price >= close_price_in_last_idx_in_window * (1.0 + float(margin_rate) / 100.0):
                target = 1
            y[idx].append(target)
            
            idx_coin += 1
           
    y = np.asarray(y)
    
    return y

def Load_Dataset_X(dir_path, time_unit, window_size, gap, margin_rate):
    key_name_X = "X_" + \
                    str(time_unit) + "_" + \
                    str(window_size) + "_" + \
                    str(gap) + "_" + \
                    str(margin_rate)
                    

    with open(dir_path + key_name_X + ".pickle", 'rb') as handle:
        b_x = pickle.load(handle)
    return b_x
    
def Load_Dataset_y(dir_path, time_unit, window_size, gap, margin_rate):
    key_name_y = "y_" + \
                    str(time_unit) + "_" + \
                    str(window_size) + "_" + \
                    str(gap) + "_" + \
                    str(margin_rate)
                    

    with open(dir_path + key_name_y + ".pickle", 'rb') as handle:
        b_y = pickle.load(handle)
    return b_y

In [3]:
# Create Dataset
data_files_dir = "/Users/dohyung/OneDrive/2018-RNN/RNN_python/AWS_dataset/"
dataset_dir_path_tuple_type = "./dataset_pickle_tuple_type/"
dataset_dir_path_numpy_type = "./dataset_pickle_numpy.ndarray_type)/"
coin_list = ["KRW", "BTC", "ETH", "XRP", "BCH", "LTC", "DASH", "ETC"]
start_date = "2017-08-04 21:40:00"
end_date = "2018-08-20 23:50:00"
# time_unit = [10,30,60]     # candle stick minutes
# window_size = [10,25,50,75,100]  # Unit: num. of candle sticks
# gap = [1,2,3]            # Unit: num. of candle sticks
# margin_rate = [0.1,0.25,0.5]  # Unit: percent

# for slave04
#time_unit = [10,30,60]     # candle stick minutes
#window_size = [10,25,50]  # Unit: num. of candle sticks
#gap = [1,2,3]            # Unit: num. of candle sticks
#margin_rate = [0.1,0.25,0.5]  # Unit: percent

# for slave05
# time_unit = [10,30,60]     # candle stick minutes
# window_size = [75,100]  # Unit: num. of candle sticks
# gap = [1,2,3]            # Unit: num. of candle sticks
# margin_rate = [0.1,0.25,0.5]  # Unit: percent

In [ ]:
def f1_score(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [6]:
def input_reshape(X_train_data, X_test_data, n_steps, n_coins, n_price):
    X_train_reshape = X_train_data.reshape(
        -1, 
        n_steps, 
        n_coins * n_price
    )
    X_test_reshape = X_test_data.reshape(
        -1, 
        n_steps, 
        n_coins * n_price
    )
    return X_train_reshape, X_test_reshape

def onehottify(x, n=None, dtype=np.int):
    """1-hot encode x with the max value n (computed from data if n is None)."""
    x = np.asarray(x)
    n = np.max(x) + 1 if n is None else n
    return np.eye(n, dtype=dtype)[x]

def Start_Model(pickle_load_dir_path, data_files_dir, epochs, pickle_result_dir_path, MODEL, idx_time_unit, idx_window_size, idx_gap, idx_margin_rate, _TEST, _ENHANCE, _GPU, n_jobs, machine, Internet_connection):
    X = {}
    y = {}

    key_name_X = "X_"
    key_name_y = "y_"

    key_name_X += str(idx_time_unit) + "_" + str(idx_window_size) + "_" + str(idx_gap) + "_" + str(idx_margin_rate)
    key_name_y += str(idx_time_unit) + "_" + str(idx_window_size) + "_" + str(idx_gap) + "_" + str(idx_margin_rate)

    X = Load_Dataset_X(pickle_load_dir_path, idx_time_unit, idx_window_size, idx_gap, idx_margin_rate)
    y = Load_Dataset_y(pickle_load_dir_path, idx_time_unit, idx_window_size, idx_gap, idx_margin_rate)

    y_single = {}
    y_single['BTC'] = y[:, 1]
    y_single['ETH'] = y[:, 2]
    y_single['XRP'] = y[:, 3]
    y_single['BCH'] = y[:, 4]
    y_single['LTC'] = y[:, 5]
    y_single['DASH'] = y[:, 6]
    y_single['ETC'] = y[:, 7]

    coin_list2 = ["BTC", "ETH", "XRP", "BCH", "LTC", "DASH", "ETC"]

    if (_TEST == False):
        for coin in coin_list2:
            if (os.path.isfile(pickle_result_dir_path + \
                              MODEL + "_" + \
                              coin + "_" + \
                              str(idx_time_unit) + "_" + \
                              str(idx_window_size) + "_" + \
                              str(idx_gap) + "_" + \
                              str(idx_margin_rate) + \
                              "_result.pickle")) is True:
                print(MODEL + "_" + \
                      coin + "_" + \
                      str(idx_time_unit) + "_" + \
                      str(idx_window_size) + "_" + \
                      str(idx_gap) + "_" + \
                      str(idx_margin_rate) + \
                      "_result.pickle FILE ALREADY EXIST.")
                continue
            else:
                y2 = onehottify(y_single[coin], n=2)

                Evaluate(pickle_load_dir_path, 
                         data_files_dir, 
                         epochs, 
                         pickle_result_dir_path,
                         _TEST, 
                         _ENHANCE,
                         coin,
                         X, y2,
                         key_name_X,
                         key_name_y,
                         idx_time_unit,
                         idx_window_size,
                         idx_gap,
                         idx_margin_rate, 
                         MODEL,
                         _GPU,
                         n_jobs,
                         machine,
                         Internet_connection)


    if (_TEST == True):
        # for test                                
        for coin in range(1):
            if (os.path.isfile(pickle_result_dir_path + \
                              "_test_" + \
                              MODEL + "_" + \
                              "BTC" + "_" + \
                              str(idx_time_unit) + "_" + \
                              str(idx_window_size) + "_" + \
                              str(idx_gap) + "_" + \
                              str(idx_margin_rate) + \
                              "_result.pickle")) is True:
                print("_test_" + \
                      MODEL + "_" + \
                      "BTC" + "_" + \
                      str(idx_time_unit) + "_" + \
                      str(idx_window_size) + "_" + \
                      str(idx_gap) + "_" + \
                      str(idx_margin_rate) + \
                      "_result.pickle FILE ALREADY EXIST.")
                continue
            else:
                y2 = onehottify(y_single['BTC'], n=2)                          

                Evaluate(pickle_load_dir_path, 
                         data_files_dir, 
                         epochs, 
                         pickle_result_dir_path, 
                         _TEST, 
                         _ENHANCE,
                         coin,
                         X, y2,
                         key_name_X,
                         key_name_y,
                         idx_time_unit,
                         idx_window_size,
                         idx_gap,
                         idx_margin_rate, 
                         MODEL,
                         _GPU,
                         n_jobs, 
                         machine,
                         Internet_connection)
    

# 저장된 pickle 파일의 데이터 구조
# tmp = {}
# tmp["10_1_1_0.1"] = {"grid_result.best_score_":{}}, {"grid_result.best_params_":{}}
# type(tmp["10_1_1_0.1"][0])
# print(tmp["10_1_1_0.1"])
# print(tmp["10_1_1_0.1"])
# print(tmp["10_1_1_0.1"][0])
# print(tmp["10_1_1_0.1"][0]['grid_result.best_score_'])

In [ ]:
def create_model_RNN(window_size, n_state_units=32, activation='softmax', optimizer='adam', init='glorot_uniform', dropout_rate=0.0, neurons=2):
    model = Sequential()
    model.add(# if문을 통해 여러 RNN모델 쓸 수 있도록 하기, SimpleRNN외에 다른 RNN모델 찾아보기
        RNN(  n_state_units, 
              input_shape=(window_size, 32),
              use_bias=True, 
              activation='tanh',
              kernel_initializer='glorot_uniform', 
              recurrent_initializer='orthogonal', 
              bias_initializer='zeros', 
              dropout=0.0,
              recurrent_dropout=0.0))
    
    model.add(Dense(units=neurons))
    model.add(Dropout(dropout_rate))
#     model.add(Dense(units=2))
    model = multi_gpu_model(model, gpus=2)

    model.compile(loss=keras.losses.categorical_crossentropy, 
                  optimizer=optimizer, 
                  metrics=["accuracy", f1_score])

    return model

def create_model_SimpleRNN(window_size, n_state_units=32, activation='softmax', optimizer='adam', init='glorot_uniform', dropout_rate=0.0, neurons=2):
    model = Sequential()
    model.add(# if문을 통해 여러 RNN모델 쓸 수 있도록 하기, SimpleRNN외에 다른 RNN모델 찾아보기
        SimpleRNN(n_state_units, 
                  input_shape=(window_size, 32),
                  use_bias=True, 
                  activation='tanh',
                  kernel_initializer='glorot_uniform', 
                  recurrent_initializer='orthogonal', 
                  bias_initializer='zeros', 
                  dropout=0.0,
                  recurrent_dropout=0.0))
    
    model.add(Dense(units=neurons))
    model.add(Dropout(dropout_rate))
#     model.add(Dense(units=2))
    model = multi_gpu_model(model, gpus=2)

    model.compile(loss=keras.losses.categorical_crossentropy, 
                  optimizer=optimizer, 
                  metrics=["accuracy", f1_score])

    return model

def create_model_LSTM(window_size, n_state_units=32, activation='softmax', optimizer='adam', init='glorot_uniform', dropout_rate=0.0, neurons=2):
    model = Sequential()
    model.add(# if문을 통해 여러 RNN모델 쓸 수 있도록 하기, SimpleRNN외에 다른 RNN모델 찾아보기
        LSTM( n_state_units, 
              input_shape=(window_size, 32),
              use_bias=True, 
              activation='tanh',
              kernel_initializer='glorot_uniform', 
              recurrent_initializer='orthogonal', 
              bias_initializer='zeros', 
              dropout=0.0,
              recurrent_dropout=0.0))
    
    model.add(Dense(units=neurons))
    model.add(Dropout(dropout_rate))
#     model.add(Dense(units=2))
    model = multi_gpu_model(model, gpus=2)

    model.compile(loss=keras.losses.categorical_crossentropy, 
                  optimizer=optimizer, 
                  metrics=["accuracy", f1_score])

    return model


def create_model_GRU(window_size, n_state_units=32, activation='softmax', optimizer='adam', init='glorot_uniform', dropout_rate=0.0, neurons=2):
    model = Sequential()
    model.add(# if문을 통해 여러 RNN모델 쓸 수 있도록 하기, SimpleRNN외에 다른 RNN모델 찾아보기
        GRU(  n_state_units, 
              input_shape=(window_size, 32),
              use_bias=True, 
              activation='tanh',
              kernel_initializer='glorot_uniform', 
              recurrent_initializer='orthogonal', 
              bias_initializer='zeros', 
              dropout=0.0,
              recurrent_dropout=0.0))
    
    model.add(Dense(units=neurons))
    model.add(Dropout(dropout_rate))
#     model.add(Dense(units=2))
    model = multi_gpu_model(model, gpus=2)

    model.compile(loss=keras.losses.categorical_crossentropy, 
                  optimizer=optimizer, 
                  metrics=["accuracy", f1_score])

    return model


In [ ]:
def create_model_RNN_non_GPU(window_size, n_state_units=32, activation='softmax', optimizer='adam', init='glorot_uniform', dropout_rate=0.0, neurons=2):
    model = Sequential()
    model.add(# if문을 통해 여러 RNN모델 쓸 수 있도록 하기, SimpleRNN외에 다른 RNN모델 찾아보기
        RNN(  n_state_units, 
              input_shape=(window_size, 32),
              use_bias=True, 
              activation='tanh',
              kernel_initializer='glorot_uniform', 
              recurrent_initializer='orthogonal', 
              bias_initializer='zeros', 
              dropout=0.0,
              recurrent_dropout=0.0))
    
    model.add(Dense(units=neurons))
    model.add(Dropout(dropout_rate))
#     model.add(Dense(units=2))
#     model = multi_gpu_model(model, gpus=2)

    model.compile(loss=keras.losses.categorical_crossentropy, 
                  optimizer=optimizer, 
                  metrics=["accuracy", f1_score])

    return model


def create_model_SimpleRNN_non_GPU(window_size, n_state_units=32, activation='softmax', optimizer='adam', init='glorot_uniform', dropout_rate=0.0, neurons=2):
    model = Sequential()
    model.add(# if문을 통해 여러 RNN모델 쓸 수 있도록 하기, SimpleRNN외에 다른 RNN모델 찾아보기
        SimpleRNN(n_state_units, 
                  input_shape=(window_size, 32),
                  use_bias=True, 
                  activation='tanh',
                  kernel_initializer='glorot_uniform', 
                  recurrent_initializer='orthogonal', 
                  bias_initializer='zeros', 
                  dropout=0.0,
                  recurrent_dropout=0.0))
    
    model.add(Dense(units=neurons))
    model.add(Dropout(dropout_rate))
#     model.add(Dense(units=2))
#     model = multi_gpu_model(model, gpus=2)

    model.compile(loss=keras.losses.categorical_crossentropy, 
                  optimizer=optimizer, 
                  metrics=["accuracy", f1_score])

    return model

def create_model_LSTM_non_GPU(window_size, n_state_units=32, activation='softmax', optimizer='adam', init='glorot_uniform', dropout_rate=0.0, neurons=2):
    model = Sequential()
    model.add(# if문을 통해 여러 RNN모델 쓸 수 있도록 하기, SimpleRNN외에 다른 RNN모델 찾아보기
        LSTM( n_state_units, 
              input_shape=(window_size, 32),
              use_bias=True, 
              activation='tanh',
              kernel_initializer='glorot_uniform', 
              recurrent_initializer='orthogonal', 
              bias_initializer='zeros', 
              dropout=0.0,
              recurrent_dropout=0.0))
    
    model.add(Dense(units=neurons))
    model.add(Dropout(dropout_rate))
#     model.add(Dense(units=2))
#     model = multi_gpu_model(model, gpus=2)

    model.compile(loss=keras.losses.categorical_crossentropy, 
                  optimizer=optimizer, 
                  metrics=["accuracy", f1_score])

    return model


def create_model_GRU_non_GPU(window_size, n_state_units=32, activation='softmax', optimizer='adam', init='glorot_uniform', dropout_rate=0.0, neurons=2):
    model = Sequential()
    model.add(# if문을 통해 여러 RNN모델 쓸 수 있도록 하기, SimpleRNN외에 다른 RNN모델 찾아보기
        GRU(  n_state_units, 
              input_shape=(window_size, 32),
              use_bias=True, 
              activation='tanh',
              kernel_initializer='glorot_uniform', 
              recurrent_initializer='orthogonal', 
              bias_initializer='zeros', 
              dropout=0.0,
              recurrent_dropout=0.0))
    
    model.add(Dense(units=neurons))
    model.add(Dropout(dropout_rate))
#     model.add(Dense(units=2))
#     model = multi_gpu_model(model, gpus=2)

    model.compile(loss=keras.losses.categorical_crossentropy, 
                  optimizer=optimizer, 
                  metrics=["accuracy", f1_score])

    return model


In [7]:
def Evaluate(pickle_load_dir_path, 
             data_files_dir, 
             epochs, 
             pickle_result_dir_path,
             _TEST, 
             _ENHANCE, 
             coin, 
             X, y2, 
             key_name_X,
             key_name_y,
             idx_time_unit,
             idx_window_size,
             idx_gap,
             idx_margin_rate, 
             MODEL,
             _GPU,
             n_jobs, 
             machine,
             Internet_connection):

    X_train, X_test, y_train, y_test = train_test_split(X, y2, test_size=0.1, random_state=42)
#     print("X_train.shape")
#     print(X_train.shape)
#     print("y_train.shape")
#     print(y_train.shape)
#     print()
#     print("X_test.shape")
#     print(X_test.shape)
#     print("y_test.shape")
#     print(y_test.shape)
#     print()

    n_coins = 8
    n_price = 4
    n_steps = idx_window_size # 원래 100이었음. reshape 문제 때문에 수정함

    X_train_2 = X_train.transpose([0, 2, 1, 3])
    X_test_2 = X_test.transpose([0, 2, 1, 3])
#     print("X_train_2.shape")
#     print(X_train_2.shape)
#     print("X_test_2.shape")
#     print(X_test_2.shape)
#     print()

    X_train_3 = X_train_2.reshape([X_train.shape[0], n_steps, n_coins * n_price])
    X_test_3 = X_test_2.reshape([X_test.shape[0], n_steps, n_coins * n_price])
#     print("X_train_3.shape")
#     print(X_train_3.shape)
#     print("X_test_3.shape")
#     print(X_test_3.shape)
#     print()

    if (_TEST==True and _ENHANCE==False):
        param_grid = {'window_size' : [n_steps], 
                      'n_state_units': [100],
                      'activation': ['relu'], 
                      'optimizer': ['rmsprop'], #sgd 추가
                      'init': ['glorot_uniform'], #he 추가
                      'batch_size': [2048]}

    elif (_TEST==False and _ENHANCE==False):
        param_grid = {'window_size' : [n_steps], 
                      'n_state_units': [40, 80, 160],
                      'activation': ['relu', 'softmax'], 
                      'optimizer': ['rmsprop', 'adam'], #sgd 추가
                      'init': ['glorot_uniform', 'uniform', 'he_uniform'], #he 추가
                      'batch_size': [64,128,256]}
        
    elif (_TEST==False and _ENHANCE==True):
        param_grid = {'window_size' : [], 
                      'n_state_units': [],
                      'activation': [], 
                      'optimizer': [], #sgd 추가
                      'init': [], #he 추가
                      'batch_size': [10, 50],
                      'dropout_rate':[0.0, 0.1, 0.2, 0.3, 0.4, 0.5], # after paramter select. when epochs raise..
                      'neurons':[2,10,100]}



    X_train_reshape = X_train_2.reshape([X_train.shape[0], n_steps*n_coins * n_price])
    X_test_reshape = X_test_2.reshape([X_test.shape[0], n_steps*n_coins * n_price])
#     print("X_train_reshape.shape")
#     print(X_train_reshape.shape)
#     print("X_test_reshape.shape")
#     print(X_test_reshape.shape)
#     print()

    scaler = MinMaxScaler()
    scaler.fit(X_train_reshape)
    X_train_scaled = scaler.transform(X_train_reshape)
    X_test_scaled = scaler.transform(X_test_reshape)

    X_train_scaled = X_train_scaled.reshape(-1, n_steps, n_coins * n_price)

    X_test_scaled = X_test_scaled.reshape(-1, n_steps, n_coins * n_price)

    if _GPU == True:
        if MODEL == "SimpleRNN" :
            model = KerasClassifier(build_fn=create_model_SimpleRNN, 
                                    epochs=epochs, 
#                                     batch_size=100, 
                                    verbose=True)

        elif MODEL == "LSTM":
            model = KerasClassifier(build_fn=create_model_LSTM, 
                                    epochs=epochs, 
#                                     batch_size=100, 
                                    verbose=True)

        elif MODEL == "RNN":
            model = KerasClassifier(build_fn=create_model_RNN, 
                                    epochs=epochs, 
#                                     batch_size=100, 
                                    verbose=True)

        elif MODEL == "GRU":
            model = KerasClassifier(build_fn=create_model_GRU, 
                                    epochs=epochs, 
#                                     batch_size=100, 
                                    verbose=True)
    elif _GPU == False:
        if MODEL == "SimpleRNN" :
            model = KerasClassifier(build_fn=create_model_SimpleRNN_non_GPU, 
                                    epochs=epochs, 
                                    batch_size=10, 
                                    verbose=True)

        elif MODEL == "LSTM":
            model = KerasClassifier(build_fn=create_model_LSTM_non_GPU, 
                                    epochs=epochs, 
                                    batch_size=10, 
                                    verbose=True)

        elif MODEL == "RNN":
            model = KerasClassifier(build_fn=create_model_RNN_non_GPU, 
                                    epochs=epochs, 
                                    batch_size=10, 
                                    verbose=True)

        elif MODEL == "GRU":
            model = KerasClassifier(build_fn=create_model_GRU_non_GPU, 
                                    epochs=epochs, 
                                    batch_size=10, 
                                    verbose=True)

    grid = GridSearchCV(
        estimator=model, 
        cv=5, 
        n_jobs=n_jobs, # test
        param_grid=param_grid,
        verbose=1)

    X_train_scaled, X_test_scaled = input_reshape(X_train_scaled, X_test_scaled, n_steps, n_coins, n_price)
    
    if (_TEST == True): 
        print()
        print()
        print("TEST!")
        print()
        print("----------------------")
        print("<"+MODEL+">")
        print("----------------------")
        print("__"+"BTC"+"__" + \
                "time unit: "+str(idx_time_unit) + "  |  " + \
                "window_size :"+str(idx_window_size) + "  |  " + \
                "gap :"+str(idx_gap) + "  |  " + \
                "margin_rate :"+str(idx_margin_rate) + \
                "  started.")
    elif (_TEST == False):
        print()
        print()
        print("----------------------")
        print("<"+MODEL+">")
        print("----------------------")
        print("__"+coin+"__" + \
                "time unit: "+str(idx_time_unit) + "  |  " + \
                "window_size :"+str(idx_window_size) + "  |  " + \
                "gap :"+str(idx_gap) + "  |  " + \
                "margin_rate :"+str(idx_margin_rate) + \
                "  started.")

    grid_result = grid.fit(X_train_scaled, 
                           y_train, 
                           validation_data=(X_test_scaled,y_test))
    
    print("----------------------")
    print("grid_result.score(X_test_scaled, y_test): ",grid_result.score(X_test_scaled, y_test))
    
    evaluate_result = {}
    
    if (_TEST == True): 
        test_score = grid_result.score(X_test_scaled, y_test)
        evaluate_result[MODEL + "_" + \
                      "BTC" + "_" + \
                      str(idx_time_unit) + "_" + \
                      str(idx_window_size) + "_" + \
                      str(idx_gap) + "_" + \
                      str(idx_margin_rate)] = {"MODEL":MODEL,\
                                        "Cryptocurrency":"BTC",\
#                                         "grid_result.cv_results_":grid_result.cv_results_, \
#                                         "grid_result.best_estimator_":grid_result.best_estimator_, \
                                        "Score":grid_result.cv_results_['mean_test_score'], \
                                        "Params":grid_result.cv_results_['params'],\
                                        "test_score":test_score}     
                                        
    elif (_TEST == False): 
        test_score = grid_result.score(X_test_scaled, y_test)
        evaluate_result[MODEL + "_" + \
                      coin + "_" + \
                      str(idx_time_unit) + "_" + \
                      str(idx_window_size) + "_" + \
                      str(idx_gap) + "_" + \
                      str(idx_margin_rate)] = {"MODEL":MODEL,\
                                        "Cryptocurrency":coin, \
                                        "Score":grid_result.cv_results_['mean_test_score'], \
                                        "Params":grid_result.cv_results_['params'],\
                                        "test_score":test_score} 
#     print()
#     print("evaluate result dict: ", evaluate_result)
#     print()

    # summarize results
    print()
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    print()
    # for checking pickle file exist
    print("---pickle saving..")
    if (_TEST == True):
        X = {}
        y = {}
        key_name_X = "X_"
        key_name_y = "y_"
        for idx_time_unit in time_unit:
            for idx_window_size in window_size:
                for idx_gap in gap:
                    for idx_margin_rate in margin_rate:
                        key_name_X += str(idx_time_unit) + "_" + str(idx_window_size) + "_" + str(idx_gap) + "_" + str(idx_margin_rate)
                        key_name_y += str(idx_time_unit) + "_" + str(idx_window_size) + "_" + str(idx_gap) + "_" + str(idx_margin_rate)
                        if (os.path.isfile(pickle_result_dir_path + \
                                          "_test_" + \
                                          MODEL + "_" + \
                                          "BTC" + "_" + \
                                          str(idx_time_unit) + "_" + \
                                          str(idx_window_size) + "_" + \
                                          str(idx_gap) + "_" + \
                                          str(idx_margin_rate) + \
                                          "_result.pickle")) is not True:
                            with open(pickle_result_dir_path + \
                                      "_test_" + \
                                      MODEL + "_" + \
                                      "BTC" + "_" + \
                                      str(idx_time_unit) + "_" + \
                                      str(idx_window_size) + "_" + \
                                      str(idx_gap) + "_" + \
                                      str(idx_margin_rate) + \
                                      "_result.pickle", 'wb') as handle:
                                pickle.dump(evaluate_result, handle, protocol=pickle.HIGHEST_PROTOCOL)
                                
                                # Sending Email
                                if Internet_connection == True:
                                    smtp = smtplib.SMTP('smtp.naver.com', 587)
                                    smtp.ehlo()      # say Hello
                                    smtp.starttls()  # TLS 사용시 필요
                                    smtp.login(EMAIL_ADDRESS, EMAIL_PASSWORD)

                                    msg = MIMEText(pickle_result_dir_path + \
                                                   "_test_" + \
                                                   MODEL + "_" + \
                                                   "BTC" + "_" + \
                                                   str(idx_time_unit) + "_" + \
                                                   str(idx_window_size) + "_" + \
                                                   str(idx_gap) + "_" + \
                                                   str(idx_margin_rate) + \
                                                   "_result.pickle")
                                    msg['Subject'] =   pickle_result_dir_path + \
                                                       MODEL + "_" + \
                                                       "BTC" + "_" + \
                                                       str(idx_time_unit) + "_" + \
                                                       str(idx_window_size) + "_" + \
                                                       str(idx_gap) + "_" + \
                                                       str(idx_margin_rate) + \
                                                       "_result.pickle"
                                    msg['To'] = EMAIL_ADDRESS
                                    smtp.sendmail(EMAIL_ADDRESS, EMAIL_ADDRESS, msg.as_string())

                                    smtp.quit()
                        else:
                            print("Already exist the file: ", pickle_result_dir_path + \
                                                              "_test_" + \
                                                              MODEL + "_" + \
                                                              "BTC" + "_" + \
                                                              str(idx_time_unit) + "_" + \
                                                              str(idx_window_size) + "_" + \
                                                              str(idx_gap) + "_" + \
                                                              str(idx_margin_rate) + \
                                                              "_result.pickle")

    elif (_TEST == False): 
        X = {}
        y = {}
        key_name_X = "X_"
        key_name_y = "y_"
        
        key_name_X += str(idx_time_unit) + "_" + str(idx_window_size) + "_" + str(idx_gap) + "_" + str(idx_margin_rate)
        key_name_y += str(idx_time_unit) + "_" + str(idx_window_size) + "_" + str(idx_gap) + "_" + str(idx_margin_rate)
        if (os.path.isfile(pickle_result_dir_path + \
                          MODEL + "_" + \
                          coin + "_" + \
                          str(idx_time_unit) + "_" + \
                          str(idx_window_size) + "_" + \
                          str(idx_gap) + "_" + \
                          str(idx_margin_rate) + \
                          "_result.pickle")) is not True:
            with open(pickle_result_dir_path + \
                      MODEL + "_" + \
                      coin + "_" + \
                      str(idx_time_unit) + "_" + \
                      str(idx_window_size) + "_" + \
                      str(idx_gap) + "_" + \
                      str(idx_margin_rate) + \
                      "_result.pickle", 'wb') as handle:
                pickle.dump(evaluate_result, handle, protocol=pickle.HIGHEST_PROTOCOL)

                # Sending Email
                if Internet_connection == True:
                    smtp = smtplib.SMTP('smtp.naver.com', 587)
                    smtp.ehlo()      # say Hello
                    smtp.starttls()  # TLS 사용시 필요
                    smtp.login(EMAIL_EMAIL_ADDRESS, EMAIL_PASSWORD)

                    msg = MIMEText(machine + \
                                   pickle_result_dir_path + \
                                   MODEL + "_" + \
                                   coin + "_" + \
                                   str(idx_time_unit) + "_" + \
                                   str(idx_window_size) + "_" + \
                                   str(idx_gap) + "_" + \
                                   str(idx_margin_rate) + \
                                   "_result.pickle")
                    msg['Subject'] =   machine + \
                                       pickle_result_dir_path + \
                                       MODEL + "_" + \
                                       coin + "_" + \
                                       str(idx_time_unit) + "_" + \
                                       str(idx_window_size) + "_" + \
                                       str(idx_gap) + "_" + \
                                       str(idx_margin_rate) + \
                                       "_result.pickle"
                    msg['To'] = EMAIL_ADDRESS
                    smtp.sendmail(EMAIL_ADDRESS, EMAIL_ADDRESS, msg.as_string())

                    smtp.quit()
        else:
            print("Already exist the file: ", pickle_result_dir_path + \
                                              "_test_" + \
                                              MODEL + "_" + \
                                              "BTC" + "_" + \
                                              str(idx_time_unit) + "_" + \
                                              str(idx_window_size) + "_" + \
                                              str(idx_gap) + "_" + \
                                              str(idx_margin_rate) + \
                                              "_result.pickle")
    
    print()
#     for mean, stdev, param in zip(means, stds, params):
#         print("%f (%f) with: %r" % (mean, stdev, param))
#     print()
    key_name_X = "X_"
    key_name_y = "y_"
    
    
#     return eval_result

In [8]:
def start(machine, Internet_connection, _TEST, _GPU, n_jobs, MODEL, idx_time_unit, idx_window_size, idx_gap, idx_margin_rate, epochs):
    '''
        [ATTENTION] In create_model METHOD part, need to set appropriate about GPU
        
        LINK01 -> GPU OFF
        MSI -> GPU OFF
        SLAVE04 -> GPU ON
        SLAVE05 -> GPU ON
    ''' 
    
    if machine=="slave05":
        #time_unit = [10,30,60]     # candle stick minutes
        #window_size = [25]  # Unit: num. of candle sticks
        #gap = [1]            # Unit: num. of candle sticks
        #margin_rate = [0.1]  # Unit: percent

        start_time = time.time()
        Start_Model( pickle_load_dir_path = "./dataset_pickle_tuple_type/",  
                     data_files_dir = dataset_dir_path_tuple_type, 
                     epochs=epochs, 
                     pickle_result_dir_path = "./evaluate_result/", 
                     MODEL=MODEL, 
                    idx_time_unit=idx_time_unit,
                    idx_window_size=idx_window_size, 
                    idx_gap=idx_gap, 
                    idx_margin_rate=idx_margin_rate, 
                     _TEST=False, 
                     _ENHANCE=False,
                     _GPU=True,
                     n_jobs=2,
                     machine=machine, 
                     Internet_connection=Internet_connection)
        
        end_time = time.time()
        print()
        print("TIME: ", end_time-start_time)
        Javascript('IPython.notebook.kernel.restart()')
        time.sleep(1)
        Javascript('IPython.notebook.execute_all_cells()')

    elif machine=="link-koreatech":
        #time_unit = [10]     # candle stick minutes
        #window_size = [10,25,50]  # Unit: num. of candle sticks
        #gap = [1]            # Unit: num. of candle sticks
        #margin_rate = [0.1]  # Unit: percent

        Start_Model( pickle_load_dir_path = "./dataset_pickle_tuple_type/",  
                     data_files_dir = dataset_dir_path_tuple_type, 
                     epochs=epochs, 
                     pickle_result_dir_path = "./evaluate_result/", 
                     MODEL=MODEL, 
                    idx_time_unit=idx_time_unit,
                    idx_window_size=idx_window_size, 
                    idx_gap=idx_gap, 
                    idx_margin_rate=idx_margin_rate,
                     _TEST=False, 
                     _ENHANCE=False,
                     _GPU=False,
                     n_jobs=1,
                     machine=machine, 
                     Internet_connection=Internet_connection)
        end_time = time.time()
        print()
        print("TIME: ", end_time-start_time)
        Javascript('IPython.notebook.kernel.restart()')
        time.sleep(1)
        Javascript('IPython.notebook.execute_all_cells()')

    elif machine=="slave04":
        #time_unit = [10]     # candle stick minutes
        #window_size = [75]  # Unit: num. of candle sticks
        #gap = [1]            # Unit: num. of candle sticks
        #margin_rate = [0.1]  # Unit: percent

        start_time = time.time()

        Start_Model(pickle_load_dir_path = "./dataset_pickle_tuple_type/",  
                 data_files_dir = dataset_dir_path_tuple_type, 
                 epochs=epochs, 
                 pickle_result_dir_path = "./evaluate_result/", 
                 MODEL=MODEL, 
                idx_time_unit=idx_time_unit,
                idx_window_size=idx_window_size, 
                idx_gap=idx_gap, 
                idx_margin_rate=idx_margin_rate,
                 _TEST=False, 
                 _ENHANCE=False,
                 _GPU=True,
                 n_jobs=2,
                 machine=machine,
                 Internet_connection=Internet_connection)

        end_time = time.time()
        print()
        print("TIME: ", end_time-start_time)
        Javascript('IPython.notebook.kernel.restart()')
        time.sleep(1)
        Javascript('IPython.notebook.execute_all_cells()')

    elif machine=="link01":
        #time_unit = [10]     # candle stick minutes
        #window_size = [25]  # Unit: num. of candle sticks
        #gap = [1,2,3]            # Unit: num. of candle sticks
        #margin_rate = [0.1]  # Unit: percent

        start_time = time.time()

        Start_Model(pickle_load_dir_path = "./dataset_pickle_tuple_type/",  
                 data_files_dir = dataset_dir_path_tuple_type, 
                 epochs=epochs, 
                 pickle_result_dir_path = "./evaluate_result/", 
                 MODEL=MODEL, 
                idx_time_unit=idx_time_unit,
                idx_window_size=idx_window_size, 
                idx_gap=idx_gap, 
                idx_margin_rate=idx_margin_rate,
                 _TEST=False,
                 _ENHANCE=False,
                 _GPU=False,
                 n_jobs=1,
                 machine=machine, 
                 Internet_connection=Internet_connection)

        end_time = time.time()
        print()
        print("TIME: ", end_time-start_time)
        Javascript('IPython.notebook.kernel.restart()')
        time.sleep(1)
        Javascript('IPython.notebook.execute_all_cells()')

    elif machine=="MSI":
        #time_unit = [10]     # candle stick minutes
        #window_size = [25]  # Unit: num. of candle sticks
        #gap = [1]            # Unit: num. of candle sticks
        #margin_rate = [0.1,0.25,0.5]  # Unit: percent

        start_time = time.time()

        Start_Model(pickle_load_dir_path = "./dataset_pickle_tuple_type/",  
                 data_files_dir = dataset_dir_path_tuple_type, 
                 epochs=epochs, 
                 pickle_result_dir_path = "./evaluate_result/", 
                 MODEL=MODEL, 
                idx_time_unit=idx_time_unit,
                idx_window_size=idx_window_size, 
                idx_gap=idx_gap, 
                idx_margin_rate=idx_margin_rate,
                 _TEST=False, 
                 _ENHANCE=False,
                 _GPU=False,
                 n_jobs=1,
                 machine=machine, 
                 Internet_connection=Internet_connection)

        end_time = time.time()
        print()
        print("TIME: ", end_time-start_time)
        Javascript('IPython.notebook.kernel.restart()')
        time.sleep(1)
        Javascript('IPython.notebook.execute_all_cells()')
        
    elif _TEST==True:
        
        #time_unit = [10]     # candle stick minutes
        #window_size = [10]  # Unit: num. of candle sticks
        #gap = [1]            # Unit: num. of candle sticks
        #margin_rate = [0.1]  # Unit: percent

        start_time = time.time()

        Start_Model(pickle_load_dir_path = "./dataset_pickle_tuple_type/",  
                 data_files_dir = dataset_dir_path_tuple_type, 
                 epochs=epochs, 
                 pickle_result_dir_path = "./evaluate_result/", 
                 MODEL=MODEL, 
                idx_time_unit=idx_time_unit,
                idx_window_size=idx_window_size, 
                idx_gap=idx_gap, 
                idx_margin_rate=idx_margin_rate,
                 _TEST=_TEST, 
                 _ENHANCE=False,
                 _GPU=_GPU,
                 n_jobs=1,
                 machine="test", 
                 Internet_connection=Internet_connection)

        end_time = time.time()
        print()
        print("TIME: ", end_time-start_time)
        Javascript('IPython.notebook.kernel.restart()')
        time.sleep(1)
        Javascript('IPython.notebook.execute_all_cells()')

In [10]:
# # load pickle file 
# import pickle
# b_x = pickle.load(open("./evaluate_result/_test_SimpleRNN_BTC_10_10_1_0.1_result.pickle", "rb"))
# b_x

In [11]:
# # Boost-up Acc, F1
# evaluate_result_dir_path = "./evaluate_result/acc_f1/"
# dataset_dir_path = dataset_dir_path_tuple_type 
# # dataset_dir_path = dataset_dir_path_numpy_type
# epochs = 100
# Evaluate(dataset_dir_path, data_files_dir, epochs, evaluate_result_dir_path, time_unit, window_size, gap, margin_rate)

In [12]:
model_info = {}
model_info["test"] = {"MODEL_list":["SimpleRNN", "LSTM", "GRU"],
                      "time_unit":[10], 
                      "window_size":[10],
                      "gap":[1], 
                      "margin_rate":[0.1]}
model_info["slave05"] = {"MODEL_list":["SimpleRNN", "LSTM", "GRU"],
                      "time_unit":[10,30,60], 
                      "window_size":[25],
                      "gap":[1], 
                      "margin_rate":[0.1]}
model_info["slave04"] = {"MODEL_list":["SimpleRNN", "LSTM", "GRU"],
                      "time_unit":[10], 
                      "window_size":[10,50,75],
                      "gap":[1], 
                      "margin_rate":[0.1]}
model_info["link01"] = {"MODEL_list":["SimpleRNN", "LSTM", "GRU"],
                      "time_unit":[10], 
                      "window_size":[25],
                      "gap":[2,3], 
                      "margin_rate":[0.1]}
model_info["link-koreatech"] = {"MODEL_list":["SimpleRNN", "LSTM", "GRU"],
                      "time_unit":[10], 
                      "window_size":[25],
                      "gap":[1], 
                      "margin_rate":[0.25,0.5]}

model_info["MSI"] = {"MODEL_list":["SimpleRNN", "LSTM", "GRU"],
                      "time_unit":[10], 
                      "window_size":[25],
                      "gap":[1], 
                      "margin_rate":[0.1,0.25,0.5]}

In [ ]:
Machine = "slave05"
# If Machine is "test" then, _TEST argument must be True
test = False
_GPU=True
Internet_connection=True
n_jobs=-1
epochs=20

for model in model_info[Machine]["MODEL_list"]:
    for idx_time_unit in model_info[Machine]["time_unit"]:
        for idx_window_size in model_info[Machine]["window_size"]:
            for idx_gap in model_info[Machine]["gap"]:
                for idx_margin_rate in model_info[Machine]["margin_rate"]:
                    start(machine=Machine, 
                         Internet_connection=Internet_connection, 
                         _TEST=test, 
                         _GPU=_GPU, 
                         n_jobs=n_jobs,
                         MODEL = model, 
                         idx_time_unit=idx_time_unit, 
                         idx_window_size=idx_window_size, 
                         idx_gap=idx_gap, 
                         idx_margin_rate=idx_margin_rate,
                         epochs=epochs)
                Javascript('IPython.notebook.kernel.restart()')
                time.sleep(1)
                Javascript('IPython.notebook.execute_all_cells()')

SimpleRNN_BTC_10_25_1_0.1_result.pickle FILE ALREADY EXIST.
SimpleRNN_ETH_10_25_1_0.1_result.pickle FILE ALREADY EXIST.
SimpleRNN_XRP_10_25_1_0.1_result.pickle FILE ALREADY EXIST.
SimpleRNN_BCH_10_25_1_0.1_result.pickle FILE ALREADY EXIST.
SimpleRNN_LTC_10_25_1_0.1_result.pickle FILE ALREADY EXIST.
SimpleRNN_DASH_10_25_1_0.1_result.pickle FILE ALREADY EXIST.
SimpleRNN_ETC_10_25_1_0.1_result.pickle FILE ALREADY EXIST.

TIME:  0.2713916301727295
SimpleRNN_BTC_30_25_1_0.1_result.pickle FILE ALREADY EXIST.
SimpleRNN_ETH_30_25_1_0.1_result.pickle FILE ALREADY EXIST.
SimpleRNN_XRP_30_25_1_0.1_result.pickle FILE ALREADY EXIST.


----------------------
<SimpleRNN>
----------------------
__BCH__time unit: 30  |  window_size :25  |  gap :1  |  margin_rate :0.1  started.
Fitting 5 folds for each of 108 candidates, totalling 540 fits
Train on 37488 samples, validate on 5207 samples
Epoch 1/20
Train on 37489 samples, validate on 5207 samples
Epoch 1/20
37488/37488 [==============================] -

37489/37489 [==============================] - 6s 173us/step
Train on 37489 samples, validate on 5207 samples
Epoch 1/20
   64/37489 [..............................] - ETA: 2:54 - loss: 0.7116 - acc: 0.5156 - f1_score: 0.0000e+00Train on 37489 samples, validate on 5207 samples
Epoch 1/20
37489/37489 [==============================] - 12s 319us/step - loss: 1.4137 - acc: 0.3823 - f1_score: 5.2528e-05 - val_loss: 0.6620 - val_acc: 0.3758 - val_f1_score: 0.0000e+00
Epoch 2/20
37489/37489 [==============================] - 12s 327us/step - loss: 0.6701 - acc: 0.6186 - f1_score: 0.6667 - val_loss: 0.6619 - val_acc: 0.6242 - val_f1_score: 0.6667
Epoch 2/20
37489/37489 [==============================] - 12s 317us/step - loss: 0.6653 - acc: 0.3817 - f1_score: 0.0000e+00 - val_loss: 0.6618 - val_acc: 0.3758 - val_f1_score: 0.0000e+00
Epoch 3/20
37489/37489 [==============================] - 12s 315us/step - loss: 0.6650 - acc: 0.6186 - f1_score: 0.6667 - val_loss: 0.6624 - val_acc: 0.6242 - val

 6080/37489 [===>..........................] - ETA: 10s - loss: 7.5660 - acc: 0.3824 - f1_score: 0.3811Train on 37489 samples, validate on 5207 samples
Epoch 1/20
37489/37489 [==============================] - 12s 318us/step - loss: 6.3928 - acc: 0.3825 - f1_score: 0.3816 - val_loss: 6.0547 - val_acc: 0.3758 - val_f1_score: 0.3759
Epoch 2/20
37489/37489 [==============================] - 13s 335us/step - loss: 1.3090 - acc: 0.6018 - f1_score: 0.6410 - val_loss: 0.6631 - val_acc: 0.6240 - val_f1_score: 0.6667
Epoch 2/20
37489/37489 [==============================] - 12s 317us/step - loss: 6.1658 - acc: 0.3825 - f1_score: 0.3817 - val_loss: 6.0547 - val_acc: 0.3758 - val_f1_score: 0.3759
Epoch 3/20
37489/37489 [==============================] - 12s 317us/step - loss: 0.6653 - acc: 0.6183 - f1_score: 0.6667 - val_loss: 0.6626 - val_acc: 0.6242 - val_f1_score: 0.6667
Epoch 3/20
37489/37489 [==============================] - 12s 321us/step - loss: 6.1658 - acc: 0.3825 - f1_score: 0.3817 - v

37489/37489 [==============================] - 13s 340us/step - loss: 0.8894 - acc: 0.6114 - f1_score: 0.6597 - val_loss: 0.6629 - val_acc: 0.6242 - val_f1_score: 0.6667
Epoch 2/20
37488/37488 [==============================] - 12s 313us/step - loss: 0.6651 - acc: 0.6188 - f1_score: 0.6667 - val_loss: 0.6622 - val_acc: 0.6242 - val_f1_score: 0.6667
Epoch 3/20
37489/37489 [==============================] - 12s 317us/step - loss: 0.6661 - acc: 0.6175 - f1_score: 0.6667 - val_loss: 0.6622 - val_acc: 0.6242 - val_f1_score: 0.6667
Epoch 3/20
37488/37488 [==============================] - 12s 314us/step - loss: 0.6650 - acc: 0.6188 - f1_score: 0.6667 - val_loss: 0.6634 - val_acc: 0.6242 - val_f1_score: 0.6667
Epoch 4/20
37489/37489 [==============================] - 12s 317us/step - loss: 0.6657 - acc: 0.6175 - f1_score: 0.6667 - val_loss: 0.6633 - val_acc: 0.6242 - val_f1_score: 0.6667
Epoch 4/20
37489/37489 [==============================] - 12s 315us/step - loss: 0.6657 - acc: 0.6175 - f1

37488/37488 [==============================] - 12s 322us/step - loss: 0.6652 - acc: 0.6188 - f1_score: 0.6667 - val_loss: 0.6618 - val_acc: 0.6242 - val_f1_score: 0.6667
Epoch 3/20
37489/37489 [==============================] - 12s 316us/step - loss: 9.8646 - acc: 0.6174 - f1_score: 0.6174 - val_loss: 9.9426 - val_acc: 0.6242 - val_f1_score: 0.6242
Epoch 4/20
37488/37488 [==============================] - 12s 317us/step - loss: 0.6647 - acc: 0.6188 - f1_score: 0.6667 - val_loss: 0.6632 - val_acc: 0.6242 - val_f1_score: 0.6667
Epoch 4/20
37489/37489 [==============================] - 12s 316us/step - loss: 9.8646 - acc: 0.6174 - f1_score: 0.6174 - val_loss: 9.9426 - val_acc: 0.6242 - val_f1_score: 0.6242
Epoch 5/20
37488/37488 [==============================] - 12s 320us/step - loss: 0.6649 - acc: 0.6188 - f1_score: 0.6667 - val_loss: 0.6622 - val_acc: 0.6242 - val_f1_score: 0.6667
Epoch 5/20
37489/37489 [==============================] - 12s 317us/step - loss: 9.8646 - acc: 0.6174 - f1

...

Epoch 5/20
37489/37489 [==============================] - 6s 150us/step - loss: 0.6651 - acc: 0.6186 - f1_score: 0.6667 - val_loss: 0.6622 - val_acc: 0.6242 - val_f1_score: 0.6667
Epoch 6/20
37489/37489 [==============================] - 6s 152us/step - loss: 0.6651 - acc: 0.6186 - f1_score: 0.6667 - val_loss: 0.6629 - val_acc: 0.6242 - val_f1_score: 0.6667
Epoch 7/20
 7296/37489 [====>.........................] - ETA: 3s - loss: 0.6628 - acc: 0.6235 - f1_score: 0.6667Train on 37489 samples, validate on 5207 samples
Epoch 1/20
37489/37489 [==============================] - 6s 148us/step - loss: 0.6650 - acc: 0.6186 - f1_score: 0.6667 - val_loss: 0.6621 - val_acc: 0.6242 - val_f1_score: 0.6667
Epoch 8/20
37489/37489 [==============================] - 6s 151us/step - loss: 0.6651 - acc: 0.6186 - f1_score: 0.6667 - val_loss: 0.6622 - val_acc: 0.6242 - val_f1_score: 0.6667
Epoch 9/20
37489/37489 [==============================] - 6s 151us/step - loss: 0.6650 - acc: 0.6186 - f1_score: 0.666

 4992/37489 [==>...........................] - ETA: 4s - loss: 0.6643 - acc: 0.3812 - f1_score: 0.0000e+00Train on 37489 samples, validate on 5207 samples
Epoch 1/20
37489/37489 [==============================] - 6s 153us/step - loss: 0.6649 - acc: 0.3817 - f1_score: 0.0000e+00 - val_loss: 0.6622 - val_acc: 0.3758 - val_f1_score: 0.0000e+00
Epoch 8/20
37489/37489 [==============================] - 6s 149us/step - loss: 0.6653 - acc: 0.3817 - f1_score: 0.0000e+00 - val_loss: 0.6618 - val_acc: 0.3758 - val_f1_score: 0.0000e+00
Epoch 9/20
37489/37489 [==============================] - 6s 154us/step - loss: 0.6649 - acc: 0.3817 - f1_score: 0.0000e+00 - val_loss: 0.6622 - val_acc: 0.3758 - val_f1_score: 0.0000e+00
Epoch 10/20
37489/37489 [==============================] - 7s 173us/step - loss: 0.6647 - acc: 0.3818 - f1_score: 0.0000e+00 - val_loss: 0.6628 - val_acc: 0.3758 - val_f1_score: 0.0000e+00
Epoch 11/20
37489/37489 [==============================] - 7s 177us/step - loss: 0.6649 - ac

Epoch 7/20
37489/37489 [==============================] - 6s 154us/step - loss: 0.6655 - acc: 0.3826 - f1_score: 0.0000e+00 - val_loss: 0.6619 - val_acc: 0.3758 - val_f1_score: 0.0000e+00
Epoch 8/20
37489/37489 [==============================] - 6s 148us/step - loss: 0.6657 - acc: 0.3826 - f1_score: 0.0000e+00 - val_loss: 0.6620 - val_acc: 0.3758 - val_f1_score: 0.0000e+00
Epoch 9/20
37489/37489 [==============================] - 6s 155us/step - loss: 0.6657 - acc: 0.3826 - f1_score: 0.0000e+00 - val_loss: 0.6619 - val_acc: 0.3758 - val_f1_score: 0.0000e+00
Epoch 10/20
37489/37489 [==============================] - 7s 175us/step - loss: 0.6656 - acc: 0.3826 - f1_score: 0.0000e+00 - val_loss: 0.6620 - val_acc: 0.3758 - val_f1_score: 0.0000e+00
Epoch 11/20
37488/37488 [==============================] - 30s 803us/step - loss: 0.7045 - acc: 0.6173 - f1_score: 0.6629 - val_loss: 0.6625 - val_acc: 0.6242 - val_f1_score: 0.6667
Epoch 2/20
37489/37489 [==============================] - 6s 172u

37489/37489 [==============================] - 6s 155us/step - loss: 0.6636 - acc: 0.3816 - f1_score: 0.0000e+00 - val_loss: 0.6619 - val_acc: 0.3766 - val_f1_score: 0.0000e+00
Epoch 8/20
37489/37489 [==============================] - 6s 161us/step - loss: 0.6629 - acc: 0.3810 - f1_score: 0.0000e+00 - val_loss: 0.6620 - val_acc: 0.3762 - val_f1_score: 0.0000e+00
Epoch 9/20
37489/37489 [==============================] - 7s 177us/step - loss: 0.6629 - acc: 0.3810 - f1_score: 0.0000e+00 - val_loss: 0.6626 - val_acc: 0.3766 - val_f1_score: 0.0000e+00
Epoch 10/20
37489/37489 [==============================] - 7s 199us/step - loss: 0.6628 - acc: 0.3805 - f1_score: 0.0000e+00 - val_loss: 0.6613 - val_acc: 0.3768 - val_f1_score: 0.0000e+00
Epoch 11/20
37489/37489 [==============================] - 31s 835us/step - loss: 0.9301 - acc: 0.6108 - f1_score: 0.6443 - val_loss: 0.6617 - val_acc: 0.6242 - val_f1_score: 0.6667
Epoch 2/20
37489/37489 [==============================] - 8s 210us/step - lo

37488/37488 [==============================] - 6s 164us/step - loss: 0.6643 - acc: 0.6188 - f1_score: 0.6667 - val_loss: 0.6620 - val_acc: 0.6240 - val_f1_score: 0.6667
Epoch 9/20
37488/37488 [==============================] - 7s 193us/step - loss: 0.6639 - acc: 0.6190 - f1_score: 0.6667 - val_loss: 0.6619 - val_acc: 0.6244 - val_f1_score: 0.6667
Epoch 10/20
37489/37489 [==============================] - 32s 843us/step - loss: 0.6729 - acc: 0.3877 - f1_score: 0.0000e+00 - val_loss: 0.6617 - val_acc: 0.3758 - val_f1_score: 0.0000e+00
Epoch 2/20
37488/37488 [==============================] - 7s 195us/step - loss: 0.6641 - acc: 0.6189 - f1_score: 0.6667 - val_loss: 0.6624 - val_acc: 0.6245 - val_f1_score: 0.6667
Epoch 11/20
37489/37489 [==============================] - 8s 216us/step - loss: 0.6656 - acc: 0.3825 - f1_score: 0.0000e+00 - val_loss: 0.6620 - val_acc: 0.3758 - val_f1_score: 0.0000e+00
Epoch 3/20
37488/37488 [==============================] - 8s 218us/step - loss: 0.6641 - acc

# Hyperio test

In [ ]:
# from __future__ import print_function
# import keras
# from keras.datasets import mnist
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Flatten
# from keras.layers import Conv2D, MaxPooling2D
# from keras import backend as K

# def create_model_SimpleRNN_non_GPU_test(x_train, y_train, x_val, y_val, params):
#     model = Sequential()
#     model.add(# if문을 통해 여러 RNN모델 쓸 수 있도록 하기, SimpleRNN외에 다른 RNN모델 찾아보기
#         SimpleRNN(
# #             n_state_units, 
#                   params['first_neuron'],
#                   input_dim=x_train.shape[2], # [dataset 크기, 윈도우 사이즈, 32(코인개수*OLHC)]
# #                   input_shape=(window_size, 32),
# #                   use_bias=True, 
#                   #activation='relu'
#         ))
# #                   kernel_initializer='glorot_uniform', 
# #                   recurrent_initializer='orthogonal', 
# #                   bias_initializer='zeros', 
# #                   dropout=0.0,
# #                   recurrent_dropout=0.0))
    
# #     model.add(Dense(units=neurons))
# #     model.add(Dropout(dropout_rate))
        
#     model.add(Dropout(params['dropout']))
#     model.add(Dense(y_train.shape[1],
#                     activation=params['last_activation']))
        
# #     model.add(Dense(units=2))
# #     model = multi_gpu_model(model, gpus=2)
# #     model.compile(optimizer=params['optimizer'](lr=lr_normalizer(params['lr'], params['optimizer'])),
# #                   loss=params['loss'],
# #                   metrics=['acc'])
#     model = multi_gpu_model(model, gpus=2)
#     model.compile(optimizer=params['optimizer'](),
#                   loss=params['loss'],
#                   metrics=['acc', f1_score])
    
#     out = model.fit(x_train, y_train,
#                     batch_size=params['batch_size'],
#                     epochs=params['epochs'],
#                     verbose=1,
#                     validation_data=[x_val, y_val])
# #                     callbacks=early_stopper(params['epochs'], mode='strict'))
    
#     return out

In [ ]:
# from keras.optimizers import Adam, Nadam
# from keras.activations import softmax
# from keras.losses import categorical_crossentropy, logcosh

# pickle_load_dir_path = "./dataset_pickle_tuple_type/"
# X = {}
# y = {}


# MODEL = ["SimpleRNN"]
# idx_time_unit = 10     # candle stick minutes
# idx_window_size = 25  # Unit: num. of candle sticks
# idx_gap = 1            # Unit: num. of candle sticks
# idx_margix_rate = 0.1  # Unit: percent

# key_name_X = "X_"
# key_name_y = "y_"


# key_name_X += str(idx_time_unit) + "_" + str(idx_window_size) + "_" + str(idx_gap) + "_" + str(idx_margix_rate)
# key_name_y += str(idx_time_unit) + "_" + str(idx_window_size) + "_" + str(idx_gap) + "_" + str(idx_margix_rate)

# X = Load_Dataset_X(pickle_load_dir_path, idx_time_unit, idx_window_size, idx_gap, idx_margix_rate)
# y = Load_Dataset_y(pickle_load_dir_path, idx_time_unit, idx_window_size, idx_gap, idx_margix_rate)

# y_single = {}
# y_single['BTC'] = y[:, 1]
# y_single['ETH'] = y[:, 2]
# y_single['XRP'] = y[:, 3]
# y_single['BCH'] = y[:, 4]
# y_single['LTC'] = y[:, 5]
# y_single['DASH'] = y[:, 6]
# y_single['ETC'] = y[:, 7]

# coin_list2 = ["BTC", "ETH", "XRP", "BCH", "LTC", "DASH", "ETC"]

# y2 = onehottify(y_single['BTC'], n=2) 
# #                         for coin in coin_list2:
# #                             print("y_single["+coin+"]"+".shape")
# #                             print(y_single[coin].shape)
# #                             print()




# X_train, X_test, y_train, y_test = train_test_split(X, y2, test_size=0.1, random_state=42)
# #     print("X_train.shape")
# #     print(X_train.shape)
# #     print("y_train.shape")
# #     print(y_train.shape)
# #     print()
# #     print("X_test.shape")
# #     print(X_test.shape)
# #     print("y_test.shape")
# #     print(y_test.shape)
# #     print()

# n_coins = 8
# n_price = 4
# n_steps = idx_window_size # 원래 100이었음. reshape 문제 때문에 수정함

# X_train_2 = X_train.transpose([0, 2, 1, 3])
# X_test_2 = X_test.transpose([0, 2, 1, 3])
# #     print("X_train_2.shape")
# #     print(X_train_2.shape)
# #     print("X_test_2.shape")
# #     print(X_test_2.shape)
# #     print()

# X_train_3 = X_train_2.reshape([X_train.shape[0], n_steps, n_coins * n_price])
# X_test_3 = X_test_2.reshape([X_test.shape[0], n_steps, n_coins * n_price])
# #     print("X_train_3.shape")
# #     print(X_train_3.shape)
# #     print("X_test_3.shape")
# #     print(X_test_3.shape)
# #     print()

# X_train_reshape = X_train_2.reshape([X_train.shape[0], n_steps*n_coins * n_price])
# X_test_reshape = X_test_2.reshape([X_test.shape[0], n_steps*n_coins * n_price])
# #     print("X_train_reshape.shape")
# #     print(X_train_reshape.shape)
# #     print("X_test_reshape.shape")
# #     print(X_test_reshape.shape)
# #     print()

# scaler = MinMaxScaler()
# scaler.fit(X_train_reshape)
# X_train_scaled = scaler.transform(X_train_reshape)
# X_test_scaled = scaler.transform(X_test_reshape)

# X_train_scaled = X_train_scaled.reshape(-1, n_steps, n_coins * n_price)

# X_test_scaled = X_test_scaled.reshape(-1, n_steps, n_coins * n_price)

# X_train_scaled, X_test_scaled = input_reshape(X_train_scaled, X_test_scaled, n_steps, n_coins, n_price)

                        
# p = {'lr': (0.1, 0.01, 0.001),
#      'first_neuron':[4, 8, 16, 32, 64, 128],
#      'batch_size': [64,128,256],
#      'epochs': [100],
#      'activation':['relu', 'softmax'],
#      'dropout': (0, 0.40, 10),
#      'optimizer': [Adam, Nadam],
#      'kernel_initializer':['glorot_uniform', 'uniform', 'he_uniform'],
#      'recurrent_initializer':['orthogonal'], 
#      'bias_initializer':['zeros'],
#      'loss': ['categorical_crossentropy', 'logcosh'],
#      'last_activation': ['softmax'],
#      'weight_regulizer':[None],
#      'emb_output_dims': [None]}

# import hyperio as hy
# h = hy.Hyperio(X_train_scaled, y_train, 
#                params=p, 
#                dataset_name='coin', 
#                experiment_no='1', 
#                model=create_model_SimpleRNN_non_GPU_test,
#                # create_model_SimpleRNN_non_GPU_test(x_train, y_train, x_val, y_val, params)
#                grid_downsample=.01)

In [ ]:
# h

In [ ]:
# h.result

In [2]:
# load pickle file 
import pickle
b_x = pickle.load(open("./paramter/10_10_1_0.1/SimpleRNN_BCH_10_10_1_0.1_result.pickle", "rb"))
b_x

FileNotFoundError: [Errno 2] No such file or directory: './paramter/10_10_1_0.1/SimpleRNN_BCH_10_10_1_0.1_result.pickle'